In [134]:
from IPython import get_ipython
from IPython.display import display

In [136]:
!pip install google-cloud-aiplatform
!pip install langchain_google_vertexai

In [137]:
!pip install langchain_huggingface google-cloud-aiplatform
!pip install langchain_community
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from google.cloud import aiplatform
# from langchain.llms import VertexAI #Depriciated
from langchain_google_vertexai import VertexAI
from langchain.evaluation import load_evaluator

In [138]:
from google.colab import auth
auth.authenticate_user()

In [139]:
PROJECT_ID = "silent-hook-454813-t3"  # GCP project ID
LOCATION = "us-central1"      # GCP location
try:
    aiplatform.init(project=PROJECT_ID, location=LOCATION)
except Exception as e:
    print(f"Error initializing Vertex AI: {e}")
    print("Please ensure your PROJECT_ID and LOCATION are correctly set, and that you have the necessary permissions.")

    raise

In [140]:
text = """
Canadian Prime Minister Justin Trudeau has reportedly acknowledged the presence of Khalistan supporters in the country but emphasised that they do not represent Sikh community as a whole.

News reports said that Trudeau made these comments while addressing the Indian diaspora at Ottawa’s Parliament Hill during last week's Diwali celebrations.

“There are many supporters of Khalistan in Canada, but they do not represent the Sikh Community as a whole. There are supporters of the Modi government in Canada, but they do not represent all Hindu Canadians as a whole,” he was quoted as saying.

Trudeau further noted that while there are supporters of Prime Minister Narendra Modi’s government in Canada, they do not represent all Hindu Canadians.

The relations between India and Canada came under severe strain following Trudeau's allegations in September last year of a “potential” involvement of Indian agents in Khalistan extremist Hardeep Singh Nijjar's killing, an allegation rejected by India as “absurd.”
"""

In [141]:
template = """Provide a concise summary of below document, focusing on its main purpose, key points, and any essential conclusions or recommendations. Outline the central themes or sections and highlight any critical information or data presented. Keep the summary brief, prioritizing clarity and readability, and avoid including minor details unless necessary to understand the overall message.
{text}

Answer: """

prompt = PromptTemplate.from_template(template)

In [142]:
try:
    llm = VertexAI(
    model_name="gemini-1.5-pro",
    max_output_tokens=512,
    temperature=0.2,
    )
except Exception as e:
    print(f"Error initializing Vertex AI LLM: {e}")
    print(
        "Please check if the model name is correct and available in your region.  Also, ensure that you have the"
        " appropriate permissions to access the model."
    )
    raise

In [143]:
llm_chain = prompt | llm

In [144]:
try:
    print("Single Summary:")
    print(llm_chain.invoke({"text": text}))
    print(prompt)
except Exception as e:
    print(f"Error during summarization: {e}")
    print("Check your prompt and input text.  Also, ensure the LLM is correctly configured.")
    raise

Single Summary:
Canadian Prime Minister Justin Trudeau acknowledged the presence of both Khalistan supporters and Narendra Modi supporters in Canada, emphasizing that neither group represents the entirety of Sikh or Hindu Canadians, respectively. This statement, made during Diwali celebrations, comes after strained relations between India and Canada following Trudeau's allegation of potential Indian agent involvement in the death of Khalistan extremist Hardeep Singh Nijjar, an allegation India deemed absurd. 

Essentially, Trudeau attempts to navigate a nuanced political landscape by recognizing the diversity of viewpoints within Canadian communities while aiming to ease diplomatic tensions with India. 

input_variables=['text'] input_types={} partial_variables={} template='Provide a concise summary of below document, focusing on its main purpose, key points, and any essential conclusions or recommendations. Outline the central themes or sections and highlight any critical information 

In [145]:
combine_prompt = """
Combine the following summaries into a single, cohesive summary that accurately represents the overall document. Integrate the main points from each chunk to reflect the document’s core themes, purpose, and conclusions. Keep the final summary concise and logically organized, eliminating any redundant or less important details to ensure a clear, unified overview.
{text}

Answer:"""

merge_prompt = PromptTemplate.from_template(combine_prompt)

In [146]:
merge_prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nCombine the following summaries into a single, cohesive summary that accurately represents the overall document. Integrate the main points from each chunk to reflect the document’s core themes, purpose, and conclusions. Keep the final summary concise and logically organized, eliminating any redundant or less important details to ensure a clear, unified overview.\n{text}\n\nAnswer:')

In [147]:
def getCombinedPrompt(explanationList, iteration, index, final=0):
    context1 = ''
    for j, response1 in enumerate(explanationList):
        context1 = context1 + "\nSummary of Chunk " + str(j + 1) + ":\n" + response1.strip() + '\n'
    return context1

In [148]:
def getCombinedLLMResponse(prompt, iteration, index, final=0):
    llm_chain = merge_prompt | llm
    try:
        response = llm_chain.invoke({"text": prompt})
        if response is not None:
            return response
    except Exception as e:
        print(f"Error in getCombinedLLMResponse: {e}")
        raise

In [149]:
def getLLMResponse(text):
    llm_chain = prompt | llm
    try:
        response = llm_chain.invoke({"text": text})
        return response
    except Exception as e:
        print(f"Error in getLLMResponse: {e}")
        raise

In [150]:
def getSummaryOfSummariesTest(explanationList, maxNumberOfSummaries=2, iteration=1):
    print(len(explanationList))
    llm_chunk_responses_subset = []
    if len(explanationList) > maxNumberOfSummaries:
        for ind, item in enumerate(explanationList):
            if ind < len(explanationList) - 1:
                llm_chunk_responses_subset.append([explanationList[ind], explanationList[ind + 1]])
    else:
        print(explanationList)
    return llm_chunk_responses_subset

In [151]:
getSummaryOfSummariesTest(['a', 'b', 'c', 'd', 'e'], maxNumberOfSummaries=2, iteration=1)

5


[['a', 'b'], ['b', 'c'], ['c', 'd'], ['d', 'e']]

In [152]:
def getSummaryOfSummaries(explanationList, maxNumberOfSummaries=2, iteration=1):
    print(len(explanationList))
    llm_chunk_responses_subset = []
    if len(explanationList) > maxNumberOfSummaries:
        for ind, item in enumerate(explanationList):
            if ind < len(explanationList) - 1:
                llm_chunk_responses_subset.append([explanationList[ind], explanationList[ind + 1]])
        llm_combined_prompts = []
        llm_combined_explanation = []
        for k, expList in enumerate(llm_chunk_responses_subset):
            pmt = getCombinedPrompt(expList, iteration, k + 1)
            try:
                res = getCombinedLLMResponse(pmt, iteration, k + 1)
                llm_combined_prompts.append(pmt)
                llm_combined_explanation.append(res)
            except Exception as e:
                print(f"Error in getSummaryOfSummaries (inner loop): {e}")
                raise

        if len(llm_combined_explanation) > maxNumberOfSummaries:
            getSummaryOfSummaries(llm_combined_explanation, maxNumberOfSummaries=2, iteration=iteration + 1)
        else:
            final_prompt = getCombinedPrompt(llm_combined_explanation, iteration, 1000, final=1)
            try:
                final_response = getCombinedLLMResponse(final_prompt, iteration, 1000, final=1)
                print('final_response', final_response)
                return final_response
            except Exception as e:
                print(f"Error in getSummaryOfSummaries (final response): {e}")
                raise
    else:
        final_prompt = getCombinedPrompt(explanationList, iteration, 1000, final=1)
        try:
            final_response = getCombinedLLMResponse(final_prompt, iteration, 1000, final=1)
            return final_response
        except Exception as e:
            print(f"Error in getSummaryOfSummaries (single chunk): {e}")
            raise

In [153]:
getLLMResponse("""The names "John Doe" for males, "Jane Doe" or "Jane Roe" for females, or "Jonnie Doe" and "Janie Doe" for children, or just "Doe" non-gender-specifically are used as placeholder names for a party whose true identity is unknown or must be withheld in a legal action, case, or discussion. The names are also used to refer to acorpse or hospital patient whose identity is unknown. This practice is widely used in the United States and Canada, but is rarely used in other English-speaking countries including the United Kingdom itself, from where the use of "John Doe" in a legal context originates. The names Joe Bloggs or John Smith are used in the UK instead, as well as in Australia and New Zealand.""")

'## Placeholder Names in Legal and Unknown Identity Cases\n\nThis document explains the use of placeholder names like "John Doe" and "Jane Doe" in legal and medical contexts. \n\n**Key Points:**\n\n* **Purpose:** These names represent individuals whose true identities are unknown or must remain confidential. This includes parties in legal cases, unidentified corpses, and hospital patients.\n* **Origin:** The practice originated in the UK but is now more common in the US and Canada.\n* **Alternatives:** Other English-speaking countries, including the UK, Australia, and New Zealand, use names like "Joe Bloggs" or "John Smith" for similar purposes.\n\n**Conclusion:**  The use of placeholder names is a practical solution for maintaining anonymity and confidentiality in specific situations. Different regions have adopted distinct naming conventions for this purpose. \n'

In [154]:
document_text = """Newtonian mechanics, or classical mechanics, was the first branch of physics to treat time and space scientifically.
It was the only physical theory describing time and space for centuries. However,changes took place at the beginning of the 20th century,
when two completely new physical theories describing time and space appeared -they were the theory of relativity and quantum mechanics.
One of the main claims of the theory of relativity is that time and space together form a single entity called "spacetime".
This is proven by the constancy of the speed of light c in vacuum with respect to any observer.
In the vicinity of large masses and also in case of extremely fast movement of masses in vacuum, time and space begin to change, i.e. transform,
in which case time slows down and the lengths of bodies shorten relative to the external observer.
In quantum mechanics, it is possible to describe the physical state of bodies (i.e. particles) only probabilistically.
This means that, for example, the physical parameters of the movement of bodies, such as speed, position or coordinate, etc.,
cannot be precisely known in advance, because so-called uncertainty relations apply. From the beginning of the 20th century until the present time,
there has been no progress beyond these notions that could be experimentally proven.
"""

In [155]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 1000
chunk_overlap = 100


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

chunks = text_splitter.split_text(document_text)

In [156]:
len(chunks)

2

In [157]:
chunk_summaries = [getLLMResponse(chunk) for chunk in chunks]

In [158]:
chunk_summaries = []
for chunk in chunks:
    try:
        chunk_summaries.append(getLLMResponse(chunk))
    except Exception as e:
        print(f"Error summarizing chunk: {e}")
        raise

len(chunk_summaries)

2

In [159]:
len(chunk_summaries)
chunk_summaries

['This document discusses the evolution of our understanding of time and space in physics. It starts by introducing **Newtonian mechanics**, the classical model that treated time and space as separate, absolute entities. \n\nThe main focus then shifts to the revolutionary changes brought about in the 20th century by:\n\n* **Theory of Relativity:** This theory, confirmed by the constant speed of light, posits that time and space are intertwined as a single entity called **spacetime**.  Furthermore, it reveals that gravity and high speeds can cause time dilation (slowing down of time) and length contraction.\n* **Quantum Mechanics:** This theory introduces the probabilistic nature of describing the state of particles, departing from the deterministic view of classical physics. \n\nEssentially, the document highlights how modern physics, through relativity and quantum mechanics, has dramatically altered our comprehension of time and space compared to the classical Newtonian perspective. \

In [160]:
summar = getSummaryOfSummaries(chunk_summaries,maxNumberOfSummaries=2, iteration = 1)

2


In [161]:
summar

"This document explores the evolution of our understanding of time and space in physics, contrasting the classical Newtonian perspective with the revolutionary insights of modern physics.  \n\nInitially, Newtonian mechanics viewed time and space as separate, absolute entities. However, the 20th century witnessed a paradigm shift with the advent of Einstein's Theory of Relativity and Quantum Mechanics. Relativity revealed the interconnected nature of time and space as a unified entity called spacetime, demonstrating how gravity and high speeds can warp time and space, leading to phenomena like time dilation and length contraction.  \n\nSimultaneously, Quantum Mechanics introduced inherent uncertainty into our understanding of the universe. The Heisenberg uncertainty principle asserts a fundamental limit to our ability to simultaneously know both the position and momentum of a particle. This inherent uncertainty in measurement, a cornerstone of quantum mechanics, implies that precise pre

In [162]:
print(getSummaryOfSummaries(chunk_summaries,maxNumberOfSummaries=2, iteration = 1))

2
This document explores the evolution of our understanding of time and space in physics, contrasting the classical Newtonian perspective with the revolutionary insights of modern physics.  

Initially, Newtonian mechanics viewed time and space as separate, absolute entities. However, the 20th century witnessed a paradigm shift with the advent of:

* **Relativity:** This theory, supported by the constant speed of light, revealed that time and space are interwoven as a single entity called spacetime. It further demonstrated that gravity and high speeds can warp spacetime, leading to phenomena like time dilation and length contraction.
* **Quantum Mechanics:** This theory introduced a probabilistic framework for describing the state of particles, challenging the deterministic worldview of classical physics.  Crucially, the uncertainty principle, a cornerstone of quantum mechanics, asserts an inherent limit to our ability to simultaneously know both the position and momentum of a particle

In [163]:
!pip install rouge-score nltk
!pip install rouge
from rouge_score import rouge_scorer
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu

In [175]:
reference_summary = """This document explores the evolution of our understanding of time and space in physics, contrasting the classical Newtonian perspective with the revolutionary insights of modern physics.

Initially, Newtonian mechanics viewed time and space as separate, absolute entities. However, the 20th century witnessed a paradigm shift with the advent of:

* **Relativity:** This theory, supported by the constant speed of light, revealed that time and space are interwoven as a single entity called spacetime. It further demonstrated that gravity and high speeds can warp spacetime, leading to phenomena like time dilation and length contraction.
* **Quantum Mechanics:** This theory introduced a probabilistic framework for describing the state of particles, challenging the deterministic worldview of classical physics.  Crucially, the uncertainty principle, a cornerstone of quantum mechanics, asserts an inherent limit to our ability to simultaneously know both the position and momentum of a particle. This fundamental uncertainty makes precise predictions about future states of physical systems impossible.

In essence, the document highlights how relativity and quantum mechanics have profoundly reshaped our comprehension of time and space, revealing a universe far more complex and counterintuitive than previously imagined.  The inherent uncertainty at the quantum level further underscores the limitations of our predictive power, marking a significant departure from the deterministic framework of classical physics.
"""

In [176]:
rouge = Rouge()
scores = rouge.get_scores(summar, reference_summary)

In [178]:
print("ROUGE Scores:")
print(scores)

ROUGE Scores:
[{'rouge-1': {'r': 0.6312056737588653, 'p': 0.7542372881355932, 'f': 0.6872586822981173}, 'rouge-2': {'r': 0.485, 'p': 0.563953488372093, 'f': 0.521505371372413}, 'rouge-l': {'r': 0.6170212765957447, 'p': 0.7372881355932204, 'f': 0.6718146668541018}}]


In [179]:
reference_tokens = reference_summary.split()
candidate_tokens = summar.split()

In [181]:
bleu_score = sentence_bleu([reference_tokens], candidate_tokens)
print("BLEU Score:", bleu_score)

BLEU Score: 0.47778538716294333


In [182]:
#Generated by ChatGPT-4.o
reference_summary = """Classical mechanics, also known as Newtonian mechanics, was the first scientific approach to understanding time and space. For centuries, it remained the sole framework for describing these concepts. However, in the early 20th century, the emergence of **relativity and quantum mechanics** revolutionized this understanding.

Einstein’s **theory of relativity** introduced the concept of **spacetime**, demonstrating that time and space are interconnected. This theory also showed that time slows down and lengths contract in the presence of strong gravitational fields or at extremely high speeds.

On the other hand, **quantum mechanics** introduced a probabilistic approach to describing the physical state of particles. It established that properties like position and velocity cannot be determined with absolute precision due to the **uncertainty principle**.

Despite these groundbreaking theories, no significant advancements beyond relativity and quantum mechanics have been experimentally verified to this day."""

In [183]:
rouge = Rouge()
scores = rouge.get_scores(summar, reference_summary)

In [184]:
print("ROUGE Scores:")
print(scores)

ROUGE Scores:
[{'rouge-1': {'r': 0.3137254901960784, 'p': 0.2711864406779661, 'f': 0.29090908593553727}, 'rouge-2': {'r': 0.05343511450381679, 'p': 0.040697674418604654, 'f': 0.046204615553595486}, 'rouge-l': {'r': 0.2647058823529412, 'p': 0.2288135593220339, 'f': 0.24545454048099186}}]


In [185]:
#Generated by Gemini-2.0
reference_summary = """Classical mechanics, initially providing a scientific framework for understanding time and space, was the dominant physical theory for many centuries.  In the early 20th century, however, the theories of relativity and quantum mechanics emerged, leading to a revised understanding.  Relativity proposes that time and space are interconnected as "spacetime," a concept supported by the constant speed of light.  According to relativity, time and space are altered by factors such as high mass and velocity.  Quantum mechanics, conversely, describes the properties of subatomic particles in terms of probability, where parameters like speed and position are subject to inherent uncertainties.  These 20th-century theories form the basis of contemporary physics."""

In [186]:
rouge = Rouge()
scores = rouge.get_scores(summar, reference_summary)

In [187]:
print("ROUGE Scores:")
print(scores)

ROUGE Scores:
[{'rouge-1': {'r': 0.4155844155844156, 'p': 0.2711864406779661, 'f': 0.32820512342616703}, 'rouge-2': {'r': 0.06862745098039216, 'p': 0.040697674418604654, 'f': 0.05109488583728532}, 'rouge-l': {'r': 0.35064935064935066, 'p': 0.2288135593220339, 'f': 0.2769230721441157}}]
